In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pickle
import time
import scipy.stats as st
import os 
from lfimodels.balancednetwork.BalancedNetwork import BalancedNetwork

In [ ]:
m = BalancedNetwork(dim=1, first_port=8010, verbose=False, n_servers=3)

In [ ]:
inference_method = 'snpe'
filename = '1502789765221072_snpe_r1_ntrain50'
time_str = filename[:filename.find('_')]
filename.find('_')
fullname = 'data/' + filename + '.p'

In [ ]:
# load data 
with open(fullname, 'rb') as handle:
    result_dict = pickle.load(handle)

In [ ]:
# unpack values 
true_params, stats_obs, nrounds, ntrain, posterior = result_dict.values()
mean, std = posterior.values()

## Compare to true parameter 

We have generated the observed data ourselves so we do have the true parameter. The mean of the posterior should be close to it when evaluated for $x=x_{obs}$

In [ ]:
theta = np.linspace(0, 5, 1000)
post_pdf = st.norm.pdf(x=theta, loc=mean, scale=std)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(theta, post_pdf, label='$\hat{p}( theta | x=x_{obs})$')
plt.axvline(x=true_params[0], label='true theta', linestyle='--', color='C1')
plt.legend()
plt.xlabel('$R_{ee}$');
filename = time_str + '_{}_posterior_r{}_ntrain{}.pdf'.format(inference_method, nrounds, ntrain)
plt.savefig('figures/' + filename)

## Posterior predictive checking 

Generate samples from the posterior and simulate them. The resulting data should be near the observed data. 

In [ ]:
m.start_server()
# generate theta +-3, 2, 1 0 stds away from mode
thetas = [mean + i * std for i in [-3, -2, -1, 0, 1, 2, 3]]
sum_stats = []
# simulate and collect sum stats
data = m.gen(thetas)
for datum in data: 
    sum_stats.append(s.calc(datum))
m.stop_server()

In [ ]:
# plot the resulting stats with the observed stats 
plt.figure(figsize=(10, 4))
sum_stats = np.array(sum_stats).squeeze()
plt.axvline(x=mean, linestyle='--', color='C4')
plt.axvline(x=true_params, linestyle='--', color='C5')
plt.plot(thetas, sum_stats, '-o')
plt.plot(true_params, stats_obs, '*')
plt.legend(['posterior mean', 'true theta', 'ff1', 'ff2', 'ff3', 'rate mean', 'rate median'])
plt.xlabel('theta')
plt.ylabel('stats')
plt.title('Summary stats +-3 std around the posterior mean')
filename = '{}_ppch_r{}_ntrain{}.pdf'.format(inference_method, nrounds, ntrain)
plt.savefig('figures/' + filename)